In [158]:
import spacy
from spacy.lang.en.examples import sentences
import pandas as pd
import json
import numpy as np
import nltk
from nltk.stem.snowball import SnowballStemmer
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import os

#nltk.download('stopwords')

In [157]:
my_list = stopwords.words('english')

In [155]:
os.chdir(r"C:\Users\Shipt\Downloads")

df = []

def parse_obj(obj):
    for key in obj:
        if isinstance(obj[key], str):
            obj[key] = obj[key].encode('latin_1').decode('utf-8')
        elif isinstance(obj[key], list):
            obj[key] = list(map(lambda x: x if type(x) != str else x.encode('latin_1').decode('utf-8'), obj[key]))
        pass
    return obj

def load_all_messages(file):
    # Open first the first message
    file = open(file)
    
    #Here we have the decoder from messnenger
    data = json.load(file, object_hook=parse_obj)
    
    df = pd.json_normalize(data['messages'])
    
    return(df)

df1 = load_all_messages('message_1.json')
df2 = load_all_messages('message_2.json')
display(df1)


,sender_name,timestamp_ms,content,type,is_unsent,is_taken_down,bumped_message_metadata.bumped_message,bumped_message_metadata.is_bumped,reactions,photos,audio_files,videos,share.link,sticker.uri,call_duration,gifs,files
0,Josh Shipton,1665554736260,no just come Jake isnt even there,Generic,False,False,no just come Jake isnt even there,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jack Kempton,1665554279814,Do I have to ask permission for essence?,Generic,False,False,Do I have to ask permission for essence?,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jack Kempton,1665554271550,Will be back at 4,Generic,False,False,Will be back at 4,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Jack Kempton,1665554264393,Beautiful. Such a nice place,Generic,False,False,Beautiful. Such a nice place,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Jack Kempton,1665554255790,What cunt that’s so whack. Egooooo,Generic,False,False,What cunt that’s so whack. Egooooo,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Josh Shipton,1648886256969,Did you roll with Lachy,Generic,False,False,Did you roll with Lachy,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,Josh Shipton,1648886253808,Yea be said he was gonna,Generic,False,False,Yea be said he was gonna,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,Jack Kempton,1648886247245,Cool how Jake came along,Generic,False,False,Cool how Jake came along,False,"[{'reaction': '❤', 'actor': 'Josh Shipton'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,Josh Shipton,1648886246572,Awesome,Generic,False,False,Awesome,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [156]:
def clean_data(df):
    #We want a usable time stamp
    df['date_time']=pd.to_datetime(df['timestamp_ms'], unit='ms') 
    
    #Way easier to work with lower cases for text
    df['content']=df['bumped_message_metadata.bumped_message'].str.lower()
    
    #Let's not work first with every data --> Only text
    df.drop(columns=['timestamp_ms','gifs','is_unsent','photos','type','videos','audio_files','sticker.uri',
                     'call_duration','share.link','files', 'bumped_message_metadata.is_bumped', 'is_taken_down', 'bumped_message_metadata.bumped_message'],inplace=True)

    df['year']=df['date_time'].dt.year
    #df=df[df['year']==2021]
    df['hour']=df['date_time'].dt.hour
    df['weekday']=df['date_time'].dt.weekday
    
    #We can exclude some non participing people
    df=df[~df['sender_name'].isin([''])]
    
    df['content']=df.content.fillna('')
    #df=df[~df['content'].isna()].reset_index()
    
    return (df)

clean_data(df1)

,sender_name,content,reactions,date_time,year,hour,weekday
0,Josh Shipton,no just come jake isnt even there,NaN,2022-10-12 06:05:36.260,2022,6,2
1,Jack Kempton,do i have to ask permission for essence?,NaN,2022-10-12 05:57:59.814,2022,5,2
2,Jack Kempton,will be back at 4,NaN,2022-10-12 05:57:51.550,2022,5,2
3,Jack Kempton,beautiful. such a nice place,NaN,2022-10-12 05:57:44.393,2022,5,2
4,Jack Kempton,what cunt that’s so whack. egooooo,NaN,2022-10-12 05:57:35.790,2022,5,2
...,...,...,...,...,...,...,...
9995,Josh Shipton,did you roll with lachy,NaN,2022-04-02 07:57:36.969,2022,7,5
9996,Josh Shipton,yea be said he was gonna,NaN,2022-04-02 07:57:33.808,2022,7,5
9997,Jack Kempton,cool how jake came along,"[{'reaction': '❤', 'actor': 'Josh Shipton'}]",2022-04-02 07:57:27.245,2022,7,5
9998,Josh Shipton,awesome,NaN,2022-04-02 07:57:26.572,2022,7,5


In [146]:
def add_information( df): 
    #number of word per message
    df['word_number'] = df['content'].apply(lambda x: len(str(x)))
  

    return(df)


add_information(df1)

,sender_name,content,is_taken_down,bumped_message_metadata.bumped_message,reactions,word_number,date_time,year,hour,weekday
0,Josh Shipton,no just come jake isnt even there,False,no just come Jake isnt even there,NaN,33,2022-10-12 06:05:36.260,2022,6,2
1,Jack Kempton,do i have to ask permission for essence?,False,Do I have to ask permission for essence?,NaN,40,2022-10-12 05:57:59.814,2022,5,2
2,Jack Kempton,will be back at 4,False,Will be back at 4,NaN,17,2022-10-12 05:57:51.550,2022,5,2
3,Jack Kempton,beautiful. such a nice place,False,Beautiful. Such a nice place,NaN,28,2022-10-12 05:57:44.393,2022,5,2
4,Jack Kempton,what cunt that’s so whack. egooooo,False,What cunt that’s so whack. Egooooo,NaN,34,2022-10-12 05:57:35.790,2022,5,2
...,...,...,...,...,...,...,...,...,...,...
9995,Josh Shipton,did you roll with lachy,False,Did you roll with Lachy,NaN,23,2022-04-02 07:57:36.969,2022,7,5
9996,Josh Shipton,yea be said he was gonna,False,Yea be said he was gonna,NaN,24,2022-04-02 07:57:33.808,2022,7,5
9997,Jack Kempton,cool how jake came along,False,Cool how Jake came along,"[{'reaction': '❤', 'actor': 'Josh Shipton'}]",24,2022-04-02 07:57:27.245,2022,7,5
9998,Josh Shipton,awesome,False,Awesome,NaN,7,2022-04-02 07:57:26.572,2022,7,5


In [147]:
def groupby_sender(df):
    df['is_message']=1
    df['has_reac']=df['reaction_received'] >0
    filter_col = [col for col in df if col.startswith('reac_from_')]

    df_grouped=df.groupby('sender_name').agg({'word_number': ['sum', 'max','mean','median'], 
                                   'answered_himself':['sum','mean'],
                                   'reaction_received':['sum','mean'],
                                    'has_reac':['sum','mean'],
                                    'is_message':['count']})
    
    
    df_grouped_has_reacfrom=df[filter_col+['sender_name']].groupby('sender_name').sum()
    df_grouped=df_grouped.merge(df_grouped_has_reacfrom,left_index=True, right_index=True)
    return(df_grouped)

